In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from tensorflow.keras.models import load_model

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results
    
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [3]:
directory_path = r"C:\Users\victus\Documents\SignLanguage\MotionDetectionforSIBIv2\DatasetV2"

doc = []

for root, dirs, files in os.walk(directory_path):
    doc.extend(dirs)
    break 
print(doc)

actions = np.array(doc)

['aku', 'apa', 'bagaimana', 'berapa', 'di', 'F', 'halo', 'I', 'J', 'K', 'kamu', 'kapan', 'ke', 'mana', 'mereka', 'nama', 'saya', 'siapa', 'Y', 'yang']


In [4]:
model = load_model('realtimeV7.h5')

In [6]:
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.3, min_tracking_confidence=0.3) as holistic:
    while cap.isOpened():

        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            confidence = res[np.argmax(res)]
            caption = f"{actions[np.argmax(res)]} ({confidence * 100:.2f}%)"
            print(caption)
            
            predictions.append(np.argmax(res))
            
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])
                    
                    confidence = res[np.argmax(res)]
                    caption = f"{actions[np.argmax(res)]} ({confidence * 100:.2f}%)"
                    display(caption)

            if len(sentence) > 5: 
                sentence = sentence[-5:]

        height, width, channels = image.shape
        text_size, _ = cv2.getTextSize(' '.join(sentence), cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
        text_width = text_size[0]
        x = (width - text_width) // 2
        cv2.putText(image, ' '.join(sentence), (x, height - 40), 
        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

'saya (98.62%)'

<class 'mediapipe.python.solution_base.SolutionOutputs'>
saya
saya (98.39%)


'saya (98.39%)'

<class 'mediapipe.python.solution_base.SolutionOutputs'>
saya
saya (98.14%)


...

'halo (89.75%)'

<class 'mediapipe.python.solution_base.SolutionOutputs'>
halo
halo (87.78%)


'halo (87.78%)'

<class 'mediapipe.python.solution_base.SolutionOutputs'>
halo
halo (82.83%)


'halo (82.83%)'